In [0]:
# Code Courtesy : https://github.com/mnielsen

In [0]:
# Author : Sahil Kalra

In [0]:
"""
network.py
~~~~~~~~~~
A module to implement the 
1. stochastic gradient descent (SGD) learning algorithm for a feedforward 
neural network.  
2. Gradients are calculated using backpropagation.  
"""

# Standard library
import random
# Third-party libraries
import numpy as np

In [0]:
# The network class, that makes a custom neural network and calculate 
# weights and bias while training using SGD.

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  
        
        For example, if the list was [2, 3, 1] then it would be a 
        three-layer network, with 
        - the first layer containing 2 neurons, 
        - the second layer 3 neurons, and 
        - the third layer 1 neuron.  
        
        The biases and weights for the network are initialized randomly, 
        using a Gaussian distribution with mean 0, and variance 1.  
        
        Note that the first layer is assumed to be an input layer, 
        and by convention we won't set any biases for those neurons, 
        since biases are only ever used in computing the outputs from 
        later layers."""

        # Four important parameters of the network are:
        # 1. Number of layers = len(sizes)
        # 2. Size of each layer = sizes
        # 3. Bias of each neuron = random number between 0-1, for all the 
        #    neurons, except the first layer, as its input layer
        #    So if we make Network([3,8,4])
        # 4. Weights between each neuron in different layers = random number 
        #    between 0-1

        # Example:
        # nw = Network([3,8,4])
        # num_layer = 3, sizes = [3,8,4]
        # bias = arrayof([random]*8, [random]*4) as first layer i.e. input do 
        # not have bias
        # weight = arrayof ([random](3*8), [random](8*4)), as its between layers
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        # sizes[1:] --> except first layer
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        # zip(sizes[:-1], sizes[1:]) --> 0*1, 1*2, 2*3 ... layers
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    # Compute the value at a layer, provided previous layer output
    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    # Stochastic Gradient Descent
    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""

        # size of test data
        if test_data: n_test = len(test_data)
        n = len(training_data)

        # The number of epochs is a hyperparameter that defines the number times
        # that the learning algorithm will work through the entire training dataset.

        # Steps:
        # 1. loop over the epochs
        # 2. shuffle training data and make mini_batches based upon mini_batch_size
        # 3. update mini_batch using back propogation
        
        for j in xrange(epochs):
            random.shuffle(training_data)
            # imp: if mini_batch_size = 1, SGD = GD
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print ("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))
            else:
                print ("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""

        # Two lists, of same size and shape as bias and weights.
        # Purpose: In these arrays, the delta values (from back propogation) 
        # will be added, and finally the weights ans bias will be updated. 
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]

        # updation of weights and bias once complete mini batch is trained.
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))


In [23]:
# Important Intuitions

# 1. Defining a network class

# Input layer: 3, Hidden layer (8, 4), output layer:1
nw = Network([3,8,4,1])
print ("num_layers: ", nw.num_layers)
print ("sizes: ", nw.sizes)
print ("biases: ", nw.biases) # --> list of 3 arrays of size 8,4,1 respectively
print ("weights: ", nw.weights) # --> list of 3 arrays of size 3*8, 8*4, 4*1


num_layers:  4
sizes:  [3, 8, 4, 1]
biases:  [array([[-0.21228116],
       [ 1.10573055],
       [ 1.49164216],
       [ 1.24948593],
       [ 0.12242802],
       [-1.12249716],
       [ 0.15248015],
       [ 0.5585457 ]]), array([[1.08249752],
       [2.82994398],
       [1.29802282],
       [1.62374744]]), array([[-0.05874254]])]
weights:  [array([[ 0.32900952,  0.81367787, -0.06575639],
       [-0.74911032,  1.26630368, -0.25991043],
       [ 0.85709015, -1.31709583,  0.10178122],
       [-0.03685791,  0.10235747, -0.70711728],
       [-0.07290356,  1.0125548 ,  0.37904376],
       [-0.0115702 , -0.07761898, -0.74945453],
       [ 0.71728484, -0.93508882,  1.12967135],
       [-1.25333591, -0.03692718,  1.32079112]]), array([[ 0.44336782,  0.83924772,  0.60879273, -0.60382327,  1.41027814,
         0.31632938, -1.15085794, -2.09471169],
       [ 0.05384218,  1.05062021,  1.44557747, -0.40770556,  1.01173277,
        -0.13268443,  1.92650087,  0.58578396],
       [-0.23869354, -0.393

In [24]:
# Important intuitions
# 2. Feedforward is simple function
# 3. SGD.

training_data = list(range(100))
epochs = 3 # Defines the number of time training needs to be done
mini_batch_size = 20 # defines the batch size 
# (smaller the size, more time in training, closer to simple Gradient descent)
for j in range(epochs):
    random.shuffle(training_data)
    mini_batches = [
        training_data[k:k+mini_batch_size]
        for k in range(0, len(training_data), mini_batch_size)]
    print ("Total mini batch: ", len(mini_batches))
    print ("Length each mini batch: ",len(mini_batches[0]))
    print ("Mini batch: ", mini_batches)

# if mini_batch_size = 1, its same as gradient descent

Total mini batch:  5
Length each mini batch:  20
Mini batch:  [[37, 31, 41, 20, 94, 58, 35, 74, 3, 14, 34, 30, 43, 7, 92, 93, 38, 51, 24, 82], [77, 71, 44, 65, 22, 56, 29, 19, 53, 79, 86, 47, 73, 95, 46, 50, 52, 11, 90, 6], [80, 13, 75, 45, 0, 8, 15, 18, 83, 61, 68, 27, 48, 87, 69, 36, 4, 70, 91, 57], [40, 33, 67, 5, 54, 60, 78, 49, 26, 88, 1, 25, 28, 21, 32, 59, 12, 84, 76, 66], [72, 39, 17, 10, 42, 96, 55, 97, 99, 23, 98, 63, 9, 89, 16, 64, 62, 85, 81, 2]]
Total mini batch:  5
Length each mini batch:  20
Mini batch:  [[19, 90, 84, 80, 41, 0, 98, 87, 62, 93, 4, 73, 56, 52, 92, 99, 77, 70, 28, 25], [9, 79, 68, 11, 17, 33, 81, 15, 26, 94, 65, 97, 44, 30, 72, 10, 16, 22, 83, 61], [49, 36, 13, 40, 55, 82, 59, 46, 89, 60, 71, 35, 18, 20, 24, 8, 37, 34, 86, 7], [67, 95, 66, 31, 78, 54, 64, 58, 76, 85, 47, 38, 88, 69, 48, 42, 23, 51, 57, 14], [5, 91, 6, 75, 27, 53, 32, 63, 12, 45, 74, 21, 3, 29, 2, 96, 1, 43, 50, 39]]
Total mini batch:  5
Length each mini batch:  20
Mini batch:  [[0, 6, 1, 9

In [25]:
# 4. 
nabla_b = [np.zeros(b.shape) for b in nw.biases]
nabla_w = [np.zeros(b.shape) for b in nw.weights]
print (nabla_b)
print (nabla_w)

[array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]]), array([[0.],
       [0.],
       [0.],
       [0.]]), array([[0.]])]
[array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0.]])]
